In [1]:
import pandas as pd
import os
import numpy as np
from expetator.tools import read_experiment, show_heatmap, add_objectives
#from expetator.tools import prune_vectors, mojitos_to_vectors, show_pct_distribution
from expetator.monitors import mojitos
import matplotlib.pyplot as plt

### Notes
- Nancy/gros & graphite no power
- Lyon/nova

In [2]:
PATH = "../../../dataset"
from scipy.stats import trim_mean
def mojitos_to_vectors_thang(mojitos_data, knowledge, proportiontocut=.1):
    tmp = [pd.DataFrame([trim_mean(bench_run, proportiontocut=proportiontocut)],
                          columns = bench_run.columns) for bench_run in mojitos_data]
    tmp = pd.concat(tmp, ignore_index=True)
    tmp['freq'] = knowledge.fmax
    return tmp

In [3]:
def loadDataset(paths):
    #load all files
    basename = []
    for path in paths:
        if path.endswith('.zip'):
            basename.append(path)
        else:
            for file in os.listdir(path):
                if os.path.isfile(os.path.join(path,file)) and file != '.DS_Store'\
                    and os.stat(os.path.join(path,file)).st_size > 1000:
                    basename.append(os.path.join(path,file))
    #print(basename)
    #read dataset
    knowledge, mojitos_data = read_experiment(basename, only_complete=True)
    #drop row where num of columns < 46: #{7, 28, 46, 48, 50}
    numcolslist = [len(a.columns) for a in mojitos_data]
    
    knowledge = knowledge.drop(np.where(np.array(numcolslist) < 46)[0]).reset_index(drop=True)
    for i in np.where(np.array(numcolslist) < 46)[0][::-1]:
        mojitos_data.pop(i)
    
    vectors = mojitos_to_vectors_thang(mojitos_data, knowledge)
    #drop experiments have inf/ 0 in power
    exper = (set(knowledge.expe) - set(knowledge[knowledge.power == float('inf')].expe)) - set(knowledge[knowledge.power == 0].expe)
    
    knowledge = knowledge[knowledge.expe.isin(exper)]
    vectors = vectors[vectors.index.isin(knowledge.index)].reset_index(drop=True)
    knowledge = knowledge.reset_index(drop=True)
    
    #create targets
    knowledge_metric = []
    vectors_metric = []
    for metric in ['energy', 'duration', 'etp']:
        knowledge_temp = knowledge.copy()
        knowledge_temp['metric'] = metric
        _ = add_objectives(knowledge_temp, objective_name='target', group_label='expe', metric=metric, ratio=1.05, stay_is_zero=False)
        _ = add_objectives(knowledge_temp, objective_name='targetZ', group_label='expe', metric=metric, ratio=1.05, stay_is_zero=True)
        knowledge_metric.append(knowledge_temp)
        vectors_metric.append(vectors)
    knowledge_metric = pd.concat(knowledge_metric, ignore_index=True)
    vectors_metric = pd.concat(vectors_metric, ignore_index=True)
    return knowledge_metric, vectors_metric

In [4]:
paths = [PATH+'/fixedtool-Dataset/'+name for name in ['grisou','grimoire','paravance','taurus']]
#paths.extend(['../../datasets/large/0_allpct_grisou-1.nancy.grid5000.fr_1589456512.zip',
# '../../datasets/large/0_allpct_grisou-1.nancy.grid5000.fr_1589492304.zip',
# '../../datasets/large/0_allpct_grisou-1.nancy.grid5000.fr_1589554681.zip',
# '../../datasets/large/0_allpct_grisou-1.nancy.grid5000.fr_1589565839.zip',
# '../../datasets/large/0_allpct_grisou-15.nancy.grid5000.fr_1589459642.zip'])
knowledge,vectors = loadDataset(paths)

uncompleted file: ../../../dataset/fixedtool-Dataset/grisou/0_allpct_grisou-22.nancy.grid5000.fr_1606934430
error _data[0][1]) <= 4 ../../../dataset/fixedtool-Dataset/paravance/0_allpct_paravance-14.rennes.grid5000.fr_1606847676_power/paravance-14.rennes.grid5000.fr_ft-C-64_1606853007
error _data[0][1]) <= 4 ../../../dataset/fixedtool-Dataset/paravance/0_allpct_paravance-14.rennes.grid5000.fr_1606847676_power/paravance-14.rennes.grid5000.fr_ft-C-64_1606853034


In [5]:
knowledge.to_csv("../csvs/knowledge_allmetrics.csv", index=False)

In [6]:
vectors.to_csv("../csvs/newvectors_allmetrics.csv", index=False)

In [10]:
knowledge[knowledge.power == float('inf')]

,hostname,fullname,nproc,duration,startTime,endTime,fmin,fmax,hostlist,basename,expe,power,energy,etp,metric,target,targetZ


In [9]:
vectors

,cache_dtlb,cache_bpu,cache_op_prefetch,branch_misses,instructions,branch_instructions,bpf_output,ref_cpu_cycles,page_faults,cache_node,...,steal,guest,guest_nice,irxp,irxb,itxp,itxb,core0,core1,freq
0,6.396583e+07,3.299885e+07,59954.052577,2.928558e+06,4.872025e+08,6.846609e+07,0.0,1.284483e+09,0.016495,63.669072,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1200000
1,6.806504e+07,3.154514e+07,62435.386845,3.174770e+06,5.283426e+08,7.425137e+07,0.0,1.285800e+09,0.056856,97.885173,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1300000
2,8.107901e+07,3.840014e+07,67307.710684,3.161835e+06,5.375613e+08,7.632447e+07,0.0,1.239158e+09,0.002401,74.558223,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1400000
3,8.688359e+07,4.039344e+07,65662.779923,3.685309e+06,6.126359e+08,8.589864e+07,0.0,1.288549e+09,0.000000,62.218790,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1500000
4,9.345775e+07,4.500695e+07,68243.347527,3.941156e+06,6.547707e+08,9.176511e+07,0.0,1.290988e+09,0.000000,69.934066,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15445,1.120450e+08,4.413004e+07,283083.075747,1.881452e+06,4.073547e+08,7.049690e+07,0.0,5.904421e+08,11.725504,106103.558721,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2.681317e+06,2.669892e+06,1900000
15446,9.771582e+07,4.217659e+07,336889.442105,1.495939e+06,3.593532e+08,6.238076e+07,0.0,4.980004e+08,11.503860,75663.862456,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2.938713e+06,2.922680e+06,2000000
15447,7.614058e+07,3.262035e+07,227539.251416,1.143488e+06,2.631998e+08,4.585664e+07,0.0,3.467478e+08,0.589943,53829.384561,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,3.223003e+06,3.191862e+06,2100000
15448,1.313228e+08,6.467930e+07,358948.892958,2.439175e+06,5.115383e+08,8.953615e+07,0.0,6.377854e+08,2.172535,124665.668310,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,3.492710e+06,3.480183e+06,2200000


In [1]:
import numpy as np

In [25]:
alis= np.array([[1.2,3]])
blis= np.array([[1,2],[3,4]])

In [26]:
alis.shape

(1, 2)

In [27]:
blis.shape

(2, 2)

In [28]:
np.append(alis,blis,axis = 0)

array([[1.2, 3. ],
       [1. , 2. ],
       [3. , 4. ]])

In [17]:
a = []

In [18]:
a.append([1,2])

In [20]:
a.append([3,4])

In [22]:
np.array(a)

array([[1, 2],
       [3, 4]])

In [23]:
a.append(['a','v'])

In [24]:
a

[[1, 2], [3, 4], ['a', 'v']]